In [3]:
#nbi:hide_in
import geopandas as gpd
import pandas as pd
idx = pd.IndexSlice
import numpy as np

import folium
from folium.plugins import MarkerCluster
import pysal as ps
from pysal.viz import mapclassify

import ipywidgets as widgets

In [4]:
#nbi:hide_in
#Patch for Chrome courtesy of : https://github.com/python-visualization/folium/issues/812#issuecomment-555238062

import base64


def _repr_html_(self, **kwargs):
    html = base64.b64encode(self.render(**kwargs).encode('utf8')).decode('utf8')
    onload = (
        'this.contentDocument.open();'
        'this.contentDocument.write(atob(this.getAttribute(\'data-html\')));'
        'this.contentDocument.close();'
    )
    if self.height is None:
        iframe = (
            '<div style="width:{width};">'
            '<div style="position:relative;width:100%;height:0;padding-bottom:{ratio};">'
            '<iframe src="about:blank" style="position:absolute;width:100%;height:100%;left:0;top:0;'
            'border:none !important;" '
            'data-html={html} onload="{onload}" '
            'allowfullscreen webkitallowfullscreen mozallowfullscreen>'
            '</iframe>'
            '</div></div>').format
        iframe = iframe(html=html, onload=onload, width=self.width, ratio=self.ratio)
    else:
        iframe = ('<iframe src="about:blank" width="{width}" height="{height}"'
                  'style="border:none !important;" '
                  'data-html={html} onload="{onload}" '
                  '"allowfullscreen" "webkitallowfullscreen" "mozallowfullscreen">'
                  '</iframe>').format
        iframe = iframe(html=html, onload=onload, width=self.width, height=self.height)
    return iframe

folium.branca.element.Figure._repr_html_ = _repr_html_

In [5]:
#nbi:hide_in
la_county = gpd.read_file("./geographies/la_county.geojson", driver='GeoJSON')

# sf_bay = (gpd.read_file('./geographies/ne_bay.geojson')
#           .append(gpd.read_file('./geographies/sf_pen.geojson'))
#           .reset_index(drop=True))

# geo = sf_bay

In [6]:
#nbi:hide_in
bay_svc = pd.read_parquet('./processed_data/Bay_svc_Oct11.parquet')
la_svc = pd.read_parquet('./processed_data/LA_svc_Oct11.parquet')

In [7]:
#fixing typo mistake "La Metro --> LA Metro"
la_svc.index = la_svc.index.set_levels(la_svc.index.levels[2].str.replace('La', 'LA'), level=2)

In [8]:
bay_area = gpd.read_file('./geographies/bay_area.geojson', driver='GeoJSON')
bay_water = gpd.read_file('./geographies/bay_water.json')
bay_area = gpd.overlay(bay_area, bay_water, how='difference')

In [9]:
#nbi:hide_in
def select_view(df_all, agencies, service_type, geo):
    serv_types = ['am_peak', 'midday', 'pm_peak', 'evening', 'early_am', 'total']
    
    assert service_type in serv_types
    
    agency_filtered = df_all
    if 'All Agencies' not in agencies:
        agency_filtered = df_all.loc[idx[:, :, agencies], :]
        
    agency_summed = agency_filtered.groupby(level=['tract', 'covid']).sum()
    agency_summed = agency_summed[[f'{service_type}_vrh']]
    
    pre_covid = agency_summed.loc[idx[:, 0], :].reset_index(level='covid', drop=True)
    covid = agency_summed.loc[idx[:, 1], :].reset_index(level='covid', drop=True)
    difference = covid - pre_covid
    
    pre_covid = pre_covid.rename(
        columns={pre_covid.columns[0]:f'{pre_covid.columns[0]}_pre_covid'})
    covid = covid.rename(
        columns={covid.columns[0]:f'{covid.columns[0]}_covid'})
    difference = difference.rename(
        columns={difference.columns[0]:f'{difference.columns[0]}_difference'})
    joined = pre_covid.join(covid).join(difference)
    joined['pct_maintained'] = joined.iloc[:, 2] / joined.iloc[:, 0] + 1
    return geo.set_index('tract').join(joined.dropna(), how='inner')

In [10]:
#nbi:hide_in
def add_choropleth(vrh_gdf, m, classifier):
    
    vrh_gdf = vrh_gdf[vrh_gdf['pct_maintained'] != np.inf]
    vrh_gdf = vrh_gdf[vrh_gdf['pct_maintained'] <= 2.0]
    vrh_gdf['pct_maintained'] = vrh_gdf['pct_maintained'] * 100
    
    if classifier == 'Fixed':
#         threshold_scale = [0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 2.0]
        threshold_scale = [0, 20, 40, 60, 80, 100, 120, 210]
    elif classifier == 'Quantiles':
        threshold_scale = mapclassify.Quantiles(
            vrh_gdf['pct_maintained'], k = 5).bins.tolist()
        threshold_scale = [vrh_gdf['pct_maintained'].min()] + threshold_scale
#     elif classifier == 'Natural Breaks':
#         threshold_scale = mapclassify.NaturalBreaks(
#             vrh_gdf['pct_maintained'], k = 5).bins.tolist()
#         threshold_scale = [vrh_gdf['pct_maintained'].min()] + threshold_scale
#     print(threshold_scale)
    choropleth = folium.Choropleth(geo_data = vrh_gdf.reset_index().to_json(),
                                   data = vrh_gdf.reset_index(),
                    columns = ('tract', 'pct_maintained'), key_on = 'feature.properties.tract',
                    nan_fill_color = 'red', fill_color = 'YlGnBu', fill_opacity = 0.6, line_opacity = 0.2,  
                    threshold_scale = threshold_scale, legend_name='Service Maintained Post-COVID (Percentage)'
                                    )
    choropleth.add_to(m)
    return

In [15]:
from IPython.display import clear_output

def visualize_supply(Region):
    
    clear_output(wait=True)
    print('Loading...')
    
    if Region == 'LA':
        service_data = la_svc
        geo = la_county
    elif Region == 'Bay Area':
        service_data = bay_svc
        geo = bay_area
        
    serv_list = [x[:-4] for x in list(service_data.columns)] 
    disp_serv = [i.replace('_', ' ').capitalize() for i in serv_list]
    disp_serv = [i.replace('Am', 'AM').replace('Pm', 'PM').replace('am', 'AM').replace('peak', 'Peak') for i in disp_serv]
    disp_to_serv = dict(zip(disp_serv, serv_list))
    agency_list = ['All Agencies'] + list(service_data.droplevel(['tract', 'covid']).index.unique())
    
    agency_widget = widgets.SelectMultiple(
    options=agency_list,
    value=[agency_list[0]],
    #rows=10,
    description='Agencies',
    disabled=False
    )
    
    service_widget = widgets.Select(
    options=disp_serv,
    value='Total',
    #rows=10,
    description='Service Type',
    disabled=False
    )
    
    classify_widget = widgets.RadioButtons(
    options=['Quantiles', 'Fixed'],
    value='Fixed', # Defaults to 'Fixed'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Classifier:',
    disabled=False
    )
    
    def interactive_map(agencies, service_type, classifier, la=False, geo=geo):
        print('Running data query...', end='')
    #     print(service_type)
        if la:
            #convert displayed names to short names
            agencies = [disp_to_agency[agency] for agency in agencies]
        service_type = disp_to_serv[service_type]
        view = select_view(service_data, agencies, service_type, geo)
        x = geo['geometry'][0].centroid.x
        y = geo['geometry'][0].centroid.y
        m = folium.Map([y, x], zoom_start = 10)
        add_choropleth(view, m, classifier)
        print(' Done!')
        print('Drawing map...')
        clear_output(wait=True)
        display(m)
        return m
    
    w = widgets.interactive_output(
    interactive_map,
    {'agencies': agency_widget, 'service_type': service_widget,
    'classifier': classify_widget})
    ui = widgets.VBox([
        widgets.HBox([agency_widget, service_widget]), 
        classify_widget])
    
    display(ui, w);

## Visualizing Transit Service Supply During the Pandemic Response

This tool visualizes an estimate of how the supply of transit service (measured as service hours) in each Census tract has changed since the start of the COVID-19 pandemic. Using GTFS data for each operator, it compares service levels from the most recent data available before March 2020 to the most recent data available from after mid-March 2020 (as of October 11).

By default, it shows an aggregation of all transit agencies in a region for which suitable data were available, but the selection boxes allow a custom subset of operators.


In [16]:
widgets.interact(visualize_supply, Region=['LA', 'Bay Area']);

interactive(children=(Dropdown(description='Region', options=('LA', 'Bay Area'), value='LA'), Output()), _dom_…